In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.useful_func import *

In [33]:
def read_data_nmt():
    with open(r'../data/fra-eng/fra.txt','r',encoding='utf-8') as f:
        result=f.read()
    return result
#@save
def preprocess_nmt(text):
    """预处理“英语－法语”数据集"""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # 在单词和标点符号之间插入空格
    out=[' '+char if i>0 and no_space(char,text[i-1]) else char for i, char in enumerate(text)]
    return ''.join(out)

def tokenize_nmt(text,num_examples=None):
    text = preprocess_nmt(read_data_nmt())
    src,tgt=[],[]
    for i,lines in enumerate(text.split('\n')):
        if num_examples and i>=num_examples:
            break
        lines=lines.split('\t')
        if len(lines)==2:
            src.append(lines[0].split(' '))
            tgt.append(lines[1].split(' '))
    return src,tgt

def truncate_pad(content,lens,pad_content):
    content=content[:lens]
    return content+[pad_content]*(lens-len(content))

In [34]:
tokenize_nmt(preprocess_nmt(read_data_nmt()))[:20]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who', '?'],
  ['wow', '!'],
  ['fire', '!'],
  ['help', '!'],
  ['jump', '.'],
  ['stop', '!'],
  ['stop', '!'],
  ['stop', '!'],
  ['wait', '!'],
  ['wait', '!'],
  ['go', 'on', '.'],
  ['go', 'on', '.'],
  ['go', 'on', '.'],
  ['hello', '!'],
  ['hello', '!'],
  ['i', 'see', '.'],
  ['i', 'try', '.'],
  ['i', 'won', '!'],
  ['i', 'won', '!'],
  ['i', 'won', '.'],
  ['oh', 'no', '!'],
  ['attack', '!'],
  ['attack', '!'],
  ['cheers', '!'],
  ['cheers', '!'],
  ['cheers', '!'],
  ['cheers', '!'],
  ['get', 'up', '.'],
  ['go', 'now', '.'],
  ['go', 'now', '.'],
  ['go', 'now', '.'],
  ['got', 'it', '!'],
  ['got', 'it', '!'],
  ['got', 'it', '?'],
  ['got', 'it', '?'],
  ['got', 'it', '?'],
  ['hop', 'in', '.'],
  ['hop', 'in', '.'],
  ['hug', 'me', '.'],
  ['hug', 'me', '.'],
  ['i', 'fell', '.'],
  ['i', 'fell', '.'],
  ['i', 'know', '.'],
  ['i', 'left', '.'],
  ['i', 'left', '.'],
  ['i', 'lost', '.'],
  ['i', 'paid

In [48]:
torch.randn(size=[2,3,5])

tensor([[[-1.4463, -1.9586,  0.7254, -0.8196,  1.5659],
         [-0.8768, -0.8833, -0.5211,  1.0965, -0.0672],
         [-0.6081, -1.8612,  2.4023,  1.8657, -1.5129]],

        [[-0.9243, -0.3610, -0.2066, -1.1369,  0.1330],
         [ 0.5273,  1.4269, -0.0541,  2.2271,  1.4547],
         [-1.0513,  1.2229, -1.3305,  0.7374, -0.0236]]])

In [49]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence

# 输入数据：批次大小 3，序列最大长度 5，词向量维度 2
padded_sequences = torch.randn(5, 3, 2)  # shape (seq_len, batch_size, input_size)
lengths = torch.tensor([5, 3, 2], dtype=torch.long)  # 实际长度（已降序排列）

# 打包序列
packed = pack_padded_sequence(
    input=padded_sequences,
    lengths=lengths,
    batch_first=False,
    enforce_sorted=True
)

In [53]:
padded_sequences

tensor([[[-1.0982,  0.4485],
         [-0.2666, -0.3849],
         [ 0.6297, -1.2275]],

        [[-0.5882, -0.0164],
         [ 0.8204,  1.0142],
         [ 1.1350,  0.8587]],

        [[-1.3437, -0.1043],
         [-0.5417, -2.1816],
         [-0.9644, -0.8595]],

        [[ 0.2032,  0.5420],
         [-1.0521,  0.4156],
         [-0.8474,  0.9895]],

        [[-1.7379,  0.4301],
         [-0.6353, -0.2629],
         [ 0.0576, -1.3117]]])

In [75]:
mask=torch.tensor([1,0])

In [76]:
x=torch.tensor([[1,1]])

In [77]:
x.masked_fill(mask==0,0)

tensor([[1, 0]])